In [6]:
import torch
from torch_geometric.data import Data
from torch_geometric.utils import from_scipy_sparse_matrix
import scipy.sparse as sp
import numpy as np
import json
from sklearn.decomposition import PCA


adj = sp.load_npz('./data_2024/adj.npz')
feat  = np.load('./data_2024/features.npy')
labels = np.load('./data_2024/labels.npy')
splits = json.load(open('./data_2024/splits.json'))
idx_train, idx_test = splits['idx_train'], splits['idx_test']


# Dimensionality Reduction
n_components = 128
pca = PCA(n_components=n_components)
reduced_feat = pca.fit_transform(feat)


# Converting the reduced features and other arrays to torch tensors
reduced_feat = torch.tensor(reduced_feat, dtype=torch.float)
full_labels = -1 * np.ones(shape=(reduced_feat.shape[0],), dtype=np.int64)
full_labels[idx_train] = labels
labels = torch.tensor(full_labels, dtype=torch.long)



edge_index, _ = from_scipy_sparse_matrix(adj)

# Converting numpy arrays to torch tensors
# feat = torch.tensor(feat, dtype=torch.float)
# full_labels = -1 * np.ones(shape=(feat.shape[0],), dtype=np.int64)
# full_labels[idx_train] = labels
# labels = torch.tensor(full_labels, dtype=torch.long)

data = Data(x=reduced_feat, edge_index=edge_index, y=labels)
# data = Data(x=feat, edge_index=edge_index, y=labels)

train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
train_mask[idx_train] = True
test_mask[idx_test] = True
data.train_mask = train_mask
data.test_mask = test_mask


# num_train = int(len(idx_train) * 0.85)

# train_indices = idx_train[:num_train]
# val_indices = idx_train[num_train:]

# train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
# val_mask = torch.zeros(data.num_nodes, dtype=torch.bool)

# train_mask[train_indices] = True
# val_mask[val_indices] = True

# data.train_mask = train_mask
# data.val_mask = val_mask

idx_train =idx_train + idx_test

train_mask[idx_test] = True
data.train_mask = train_mask
data.train_mask.tolist().count(True)

2480

In [7]:
data

Data(x=[2480, 128], edge_index=[2, 10100], y=[2480], train_mask=[2480], test_mask=[2480])

In [8]:
set(data.y.tolist())

{-1, 0, 1, 2, 3, 4, 5, 6}

## GCN Model

In [9]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, num_node_features, num_hidden, num_classes):
        super().__init__()
        self.conv1 = GCNConv(num_node_features, num_hidden)
        self.hid1 = GCNConv(num_hidden, 16)
        self.hid2 = GCNConv(16, num_hidden)
        self.conv2 = GCNConv(num_hidden, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.hid1(x, edge_index)
        x = F.dropout(x, training=self.training)
        x = self.hid2(x, edge_index)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [17]:
from sklearn.model_selection import KFold, StratifiedKFold
import torch.nn as nn

device = torch.device('mps' if torch.cuda.is_available() else 'cpu')
k = 8
kf = StratifiedKFold(n_splits=k)
idx_train_np = np.array(idx_train)
labels = data.y.numpy()[idx_train_np]

best_accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(idx_train_np, labels)):
    print(f"Fold {fold+1}/{k}")

    model = GCN(num_node_features=data.x.shape[1], 
                num_hidden=64,
                num_classes=(data.y.max()+1).item()
               ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.09, weight_decay=5e-4)
    criterion = nn.CrossEntropyLoss(ignore_index=-1)
    
    
    data.train_mask = torch.zeros(data.y.size(0), dtype=torch.bool)
    data.val_mask = torch.zeros(data.y.size(0), dtype=torch.bool)
    data.train_mask[idx_train_np[train_idx]] = True
    data.val_mask[idx_train_np[val_idx]] = True
    
    best_val_acc = 0 
    best_model_state = None 
    
    for epoch in range(200):
        model.train()
        optimizer.zero_grad()
        out = model(data)
#         loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask], ignore_index=-1)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            model.eval()
            with torch.no_grad():
                model.eval()
                pred = model(data).argmax(dim=1)
#                 correct = (pred[data.val_mask] == data.y[data.val_mask]).sum()
#                 acc = int(correct) / int(data.val_mask.sum())
                
                valid_labels_mask = data.y[data.val_mask] != -1  # Mask to select valid labels not equal to -1
                correct_predictions = pred[data.val_mask][valid_labels_mask] == data.y[data.val_mask][valid_labels_mask]
                correct = correct_predictions.sum()
                acc = int(correct) / int(valid_labels_mask.sum())
                
                if acc > best_val_acc and acc >= 0.84:
                    best_val_acc = acc
                    best_model_state = model.state_dict()
                    
                val_loss = F.nll_loss(out[data.val_mask], data.y[data.val_mask], ignore_index=-1)
                print(f'Epoch {epoch}: Train Loss: {loss.item()}, Val Loss: {val_loss.item()}, Val Acc: {acc:.4f}')
            
    if best_model_state is not None:
        torch.save(best_model_state, f'gcn_best_fold_{fold+1}.pt')
    if best_val_acc>0:    
        best_accuracies.append(best_val_acc)
    
average_accuracy = sum(best_accuracies) / len(best_accuracies)
print(f'Average Validation Accuracy: {average_accuracy:.4f}')

Fold 1/8
Epoch 0: Train Loss: 1.950131893157959, Val Loss: 1.9227735996246338, Val Acc: 0.2903
Epoch 10: Train Loss: 0.600151777267456, Val Loss: 0.7727506160736084, Val Acc: 0.8387
Epoch 20: Train Loss: 0.14646536111831665, Val Loss: 1.2399567365646362, Val Acc: 0.8226
Epoch 30: Train Loss: 0.10245935618877411, Val Loss: 1.361914038658142, Val Acc: 0.8387
Epoch 40: Train Loss: 0.08514180034399033, Val Loss: 1.0897802114486694, Val Acc: 0.7903
Epoch 50: Train Loss: 0.07256221771240234, Val Loss: 1.0307186841964722, Val Acc: 0.8226
Epoch 60: Train Loss: 0.06738454103469849, Val Loss: 0.7299565076828003, Val Acc: 0.8226
Epoch 70: Train Loss: 0.058953724801540375, Val Loss: 0.8528133630752563, Val Acc: 0.8065
Epoch 80: Train Loss: 0.05527227371931076, Val Loss: 0.8474425077438354, Val Acc: 0.7903
Epoch 90: Train Loss: 0.04216095805168152, Val Loss: 0.9238696098327637, Val Acc: 0.7742
Epoch 100: Train Loss: 0.04192211106419563, Val Loss: 0.8398582935333252, Val Acc: 0.7903
Epoch 110: Train

Epoch 120: Train Loss: 0.052247676998376846, Val Loss: 1.503338098526001, Val Acc: 0.8065
Epoch 130: Train Loss: 0.05362817272543907, Val Loss: 1.3479056358337402, Val Acc: 0.8387
Epoch 140: Train Loss: 0.025180209428071976, Val Loss: 1.6516815423965454, Val Acc: 0.8387
Epoch 150: Train Loss: 0.040628205984830856, Val Loss: 1.4804039001464844, Val Acc: 0.8065
Epoch 160: Train Loss: 0.04629680514335632, Val Loss: 1.7349607944488525, Val Acc: 0.7742
Epoch 170: Train Loss: 0.043682731688022614, Val Loss: 1.762916088104248, Val Acc: 0.8226
Epoch 180: Train Loss: 0.035872966051101685, Val Loss: 1.8489669561386108, Val Acc: 0.8226
Epoch 190: Train Loss: 0.03677970543503761, Val Loss: 1.8239011764526367, Val Acc: 0.8387
Fold 6/8
Epoch 0: Train Loss: 1.9505119323730469, Val Loss: 1.946790337562561, Val Acc: 0.3710
Epoch 10: Train Loss: 0.40146687626838684, Val Loss: 1.4006539583206177, Val Acc: 0.8226
Epoch 20: Train Loss: 0.1241835281252861, Val Loss: 2.6854312419891357, Val Acc: 0.8226
Epoch

## APPNP Model

In [19]:
import torch
from torch_geometric.nn import APPNP
import torch.nn.functional as F
from torch_geometric.data import Data
from sklearn.model_selection import StratifiedKFold
import numpy as np
import torch.nn as nn

class APPNPNet(torch.nn.Module):
    def __init__(self, num_node_features, num_hidden, num_classes, K=10, alpha=0.1):
        super(APPNPNet, self).__init__()
        self.lin = torch.nn.Linear(num_node_features, num_hidden)
        self.appnp = APPNP(K, alpha)
        self.fc = torch.nn.Linear(num_hidden, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.lin(x))
        x = F.dropout(x, training=self.training)
        x = self.fc(x)
        return self.appnp(x, edge_index)

device = torch.device('mps' if torch.cuda.is_available() else 'cpu')
k = 9
kf = StratifiedKFold(n_splits=k)
idx_train_np = np.array(idx_train)
labels = data.y.numpy()[idx_train_np]

best_accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(idx_train_np, labels)):
    print(f"Fold {fold+1}/{k}")
    model = APPNPNet(num_node_features=data.x.shape[1], 
                     num_hidden=64,
                     num_classes=(data.y.max()+1).item(),
                     K=10,
                     alpha=0.1).to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.009, weight_decay=5e-4)
    criterion = nn.CrossEntropyLoss(ignore_index=-1)
    
    data.train_mask = torch.zeros(data.y.size(0), dtype=torch.bool)
    data.val_mask = torch.zeros(data.y.size(0), dtype=torch.bool)
    data.train_mask[idx_train_np[train_idx]] = True
    data.val_mask[idx_train_np[val_idx]] = True
    
    best_val_acc = 0
    best_model_state = None
    
    for epoch in range(200):
        model.train()
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        
        if epoch % 20 == 0:
            model.eval()
            with torch.no_grad():
                pred = model(data).argmax(dim=1)
                valid_labels_mask = data.y[data.val_mask] != -1  # Mask to select valid labels
                correct_predictions = pred[data.val_mask][valid_labels_mask] == data.y[data.val_mask][valid_labels_mask]
                correct = correct_predictions.sum()
                acc = int(correct) / int(valid_labels_mask.sum())
                
                if acc > best_val_acc and acc >= 0.87:
                    best_val_acc = acc
                    best_model_state = model.state_dict()
                    
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
                print(f'Epoch {epoch}: Train Loss: {loss.item()}, Val Loss: {val_loss.item()}, Val Acc: {acc:.4f}')
            
    if best_model_state is not None:
        torch.save(best_model_state, f'appnp_best_fold_{fold+1}.pt')
    
    if best_val_acc>0:
        best_accuracies.append(best_val_acc)

average_accuracy = sum(best_accuracies) / len(best_accuracies)
print(f'Average Validation Accuracy: {average_accuracy:.4f}')


Fold 1/9
Epoch 0: Train Loss: 1.9440314769744873, Val Loss: 1.9476426839828491, Val Acc: 0.2857
Epoch 20: Train Loss: 0.8606126308441162, Val Loss: 0.9268234372138977, Val Acc: 0.8036
Epoch 40: Train Loss: 0.321504145860672, Val Loss: 0.4543730318546295, Val Acc: 0.8929
Epoch 60: Train Loss: 0.15981808304786682, Val Loss: 0.4407646358013153, Val Acc: 0.8750
Epoch 80: Train Loss: 0.11944343894720078, Val Loss: 0.45061662793159485, Val Acc: 0.8571
Epoch 100: Train Loss: 0.09612695872783661, Val Loss: 0.39936086535453796, Val Acc: 0.8393
Epoch 120: Train Loss: 0.08140906691551208, Val Loss: 0.4857498109340668, Val Acc: 0.8393
Epoch 140: Train Loss: 0.08108352869749069, Val Loss: 0.44900423288345337, Val Acc: 0.8393
Epoch 160: Train Loss: 0.06982582807540894, Val Loss: 0.5949245095252991, Val Acc: 0.8393
Epoch 180: Train Loss: 0.06191379949450493, Val Loss: 0.4607880413532257, Val Acc: 0.8214
Fold 2/9
Epoch 0: Train Loss: 1.959958553314209, Val Loss: 1.9598253965377808, Val Acc: 0.3273
Epo

## GAT Model

In [21]:
import torch
from torch_geometric.nn import GATConv
import torch.nn.functional as F

class GAT(torch.nn.Module):
    def __init__(self, num_node_features, num_hidden, num_classes, heads=12, output_heads=1):
        super(GAT, self).__init__()
        self.conv1 = GATConv(num_node_features, num_hidden, heads=heads, dropout=0.2)
        self.conv2 = GATConv(num_hidden*heads, num_classes, heads=output_heads, concat=False, dropout=0.1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # First Graph Attention Layer
        x = F.dropout(x,training=self.training)
        x = self.conv1(x, edge_index)
        x = F.relu(x)

        # Second Graph Attention Layer
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)
    


device = torch.device('mps' if torch.cuda.is_available() else 'cpu')
k = 10

kf = StratifiedKFold(n_splits=k)
idx_train_np = np.array(idx_train)
labels = data.y.numpy()[idx_train_np]

best_accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(idx_train_np, labels)):
    print(f"Fold {fold+1}/{k}")
    model = GAT(num_node_features=data.x.shape[1], 
                num_hidden=128,
                num_classes=(data.y.max()+1).item()
               ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.009, weight_decay=5e-4)
    criterion = nn.CrossEntropyLoss(ignore_index=-1)
    
    data.train_mask = torch.zeros(data.y.size(0), dtype=torch.bool)
    data.val_mask = torch.zeros(data.y.size(0), dtype=torch.bool)
    data.train_mask[idx_train_np[train_idx]] = True
    data.val_mask[idx_train_np[val_idx]] = True
    
    best_val_acc = 0 
    best_model_state = None 
    
    for epoch in range(200):
        model.train()
        optimizer.zero_grad()
        out = model(data)
#         loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask], ignore_index=-1)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        
        if epoch % 20 == 0:
            model.eval()
            with torch.no_grad():
                model.eval()
                pred = model(data).argmax(dim=1)
                correct = (pred[data.val_mask] == data.y[data.val_mask]).sum()
                acc = int(correct) / int(data.val_mask.sum())
                
                valid_labels_mask = data.y[data.val_mask] != -1  # Mask to select valid labels not equal to -1
                correct_predictions = pred[data.val_mask][valid_labels_mask] == data.y[data.val_mask][valid_labels_mask]
                correct = correct_predictions.sum()
                acc = int(correct) / int(valid_labels_mask.sum())
                
                if acc > best_val_acc and acc >= 0.87:
                    best_val_acc = acc
                    best_model_state = model.state_dict()
                    
                val_loss = F.nll_loss(out[data.val_mask], data.y[data.val_mask], ignore_index=-1)
                print(f'Epoch {epoch}: Train Loss: {loss.item()}, Val Loss: {val_loss.item()}, Val Acc: {acc:.4f}')
            
    if best_model_state is not None:
        torch.save(best_model_state, f'gat_best_fold_{fold+1}.pt')
    if best_val_acc>0:
        best_accuracies.append(best_val_acc)

average_accuracy = sum(best_accuracies) / len(best_accuracies)
print(f'Average Validation Accuracy: {average_accuracy:.4f}')


Fold 1/10
Epoch 0: Train Loss: 1.945572018623352, Val Loss: 1.9549434185028076, Val Acc: 0.5400
Epoch 20: Train Loss: 0.31886059045791626, Val Loss: 0.4142930209636688, Val Acc: 0.9400
Epoch 40: Train Loss: 0.26370322704315186, Val Loss: 0.6633957028388977, Val Acc: 0.8800
Epoch 60: Train Loss: 0.18635545670986176, Val Loss: 0.4455018937587738, Val Acc: 0.9000
Epoch 80: Train Loss: 0.1568804532289505, Val Loss: 0.43569836020469666, Val Acc: 0.8800
Epoch 100: Train Loss: 0.12510935962200165, Val Loss: 0.5203189253807068, Val Acc: 0.8800
Epoch 120: Train Loss: 0.14253047108650208, Val Loss: 0.4069465696811676, Val Acc: 0.9000
Epoch 140: Train Loss: 0.11098465323448181, Val Loss: 0.6794502139091492, Val Acc: 0.8800
Epoch 160: Train Loss: 0.14999504387378693, Val Loss: 0.47334206104278564, Val Acc: 0.9000
Epoch 180: Train Loss: 0.16334867477416992, Val Loss: 0.6915886402130127, Val Acc: 0.8200
Fold 2/10
Epoch 0: Train Loss: 1.9455585479736328, Val Loss: 1.938031792640686, Val Acc: 0.5600
E

Epoch 40: Train Loss: 0.283584862947464, Val Loss: 0.8661600351333618, Val Acc: 0.8776
Epoch 60: Train Loss: 0.19523926079273224, Val Loss: 0.572965681552887, Val Acc: 0.8776
Epoch 80: Train Loss: 0.16296552121639252, Val Loss: 0.4113086760044098, Val Acc: 0.8776
Epoch 100: Train Loss: 0.16832558810710907, Val Loss: 0.6265553832054138, Val Acc: 0.8776
Epoch 120: Train Loss: 0.12474682182073593, Val Loss: 0.6574022173881531, Val Acc: 0.8776
Epoch 140: Train Loss: 0.14663568139076233, Val Loss: 0.8875391483306885, Val Acc: 0.8571
Epoch 160: Train Loss: 0.11579129844903946, Val Loss: 0.9596827626228333, Val Acc: 0.8776
Epoch 180: Train Loss: 0.15791785717010498, Val Loss: 0.6132438778877258, Val Acc: 0.8776
Average Validation Accuracy: 0.8950


## GSage Model

In [22]:
import torch
from torch_geometric.nn import SAGEConv
import torch.nn.functional as F

class GraphSAGE(torch.nn.Module):
    def __init__(self, num_node_features, num_hidden, num_classes):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(num_node_features, num_hidden)
        self.conv2 = SAGEConv(num_hidden, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # First GraphSAGE Layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)

        # Second GraphSAGE Layer
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

    
from sklearn.model_selection import KFold
import torch.nn as nn

device = torch.device('mps' if torch.cuda.is_available() else 'cpu')
k = 9
kf = StratifiedKFold(n_splits=k)
idx_train_np = np.array(idx_train)
labels = data.y.numpy()[idx_train_np]

best_accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(idx_train_np, labels)):
    print(f"Fold {fold+1}/{k}")
    model = GraphSAGE(num_node_features=data.x.shape[1], 
                num_hidden=128,
                num_classes=(data.y.max()+1).item()
               ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.009, weight_decay=5e-4)
    criterion = nn.CrossEntropyLoss(ignore_index=-1)
    
    data.train_mask = torch.zeros(data.y.size(0), dtype=torch.bool)
    data.val_mask = torch.zeros(data.y.size(0), dtype=torch.bool)
    data.train_mask[idx_train_np[train_idx]] = True
    data.val_mask[idx_train_np[val_idx]] = True
    
    best_val_acc = 0 
    best_model_state = None 
    
    for epoch in range(200):
        model.train()
        optimizer.zero_grad()
        out = model(data)
#         loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask], ignore_index=-1)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        
        if epoch % 20 == 0:
            model.eval()
            with torch.no_grad():
                model.eval()
                pred = model(data).argmax(dim=1)
                correct = (pred[data.val_mask] == data.y[data.val_mask]).sum()
                acc = int(correct) / int(data.val_mask.sum())
                
                valid_labels_mask = data.y[data.val_mask] != -1  # Mask to select valid labels not equal to -1
                correct_predictions = pred[data.val_mask][valid_labels_mask] == data.y[data.val_mask][valid_labels_mask]
                correct = correct_predictions.sum()
                acc = int(correct) / int(valid_labels_mask.sum())
                
                if acc > best_val_acc and acc >= 0.85:
                    best_val_acc = acc
                    best_model_state = model.state_dict()
                    
                val_loss = F.nll_loss(out[data.val_mask], data.y[data.val_mask], ignore_index=-1)
                print(f'Epoch {epoch}: Train Loss: {loss.item()}, Val Loss: {val_loss.item()}, Val Acc: {acc:.4f}')
            
    if best_model_state is not None:
        torch.save(best_model_state, f'gsage_best_fold_{fold+1}.pt')
    if best_val_acc>0:
        best_accuracies.append(best_val_acc)

average_accuracy = sum(best_accuracies) / len(best_accuracies)
print(f'Average Validation Accuracy: {average_accuracy:.4f}')



Fold 1/9
Epoch 0: Train Loss: 1.96201491355896, Val Loss: 1.9570649862289429, Val Acc: 0.3750
Epoch 20: Train Loss: 0.07552532851696014, Val Loss: 0.35585102438926697, Val Acc: 0.8750
Epoch 40: Train Loss: 0.008769199252128601, Val Loss: 0.47874197363853455, Val Acc: 0.7857
Epoch 60: Train Loss: 0.007613993715494871, Val Loss: 0.5293914675712585, Val Acc: 0.8036
Epoch 80: Train Loss: 0.010285736061632633, Val Loss: 0.5392610430717468, Val Acc: 0.8036
Epoch 100: Train Loss: 0.013431865721940994, Val Loss: 0.4552469253540039, Val Acc: 0.8036
Epoch 120: Train Loss: 0.011408250778913498, Val Loss: 0.49400249123573303, Val Acc: 0.8036
Epoch 140: Train Loss: 0.009628426283597946, Val Loss: 0.512694239616394, Val Acc: 0.8036
Epoch 160: Train Loss: 0.008534224703907967, Val Loss: 0.483083039522171, Val Acc: 0.8036
Epoch 180: Train Loss: 0.010781959630548954, Val Loss: 0.6315088272094727, Val Acc: 0.8036
Fold 2/9
Epoch 0: Train Loss: 1.9511557817459106, Val Loss: 1.9485162496566772, Val Acc: 0.

## Stacking GCN, GAT & GSage Models

In [23]:
from collections import defaultdict
from scipy.stats import mode

device = torch.device('mps' if torch.cuda.is_available() else 'cpu')
num_classes = (data.y.max() + 1).item() 


all_predictions = []

model_types = ['gcn', 'gat', 'gsage','appnp'] 
num_folds = 10

for model_type in model_types:
    for fold in range(1, num_folds + 1):
        if model_type == 'gcn':
            model = GCN(num_node_features=data.x.shape[1], 
                num_hidden=64,
                num_classes=(data.y.max()+1).item()
               ).to(device)
        elif model_type == 'gat':
            model = GAT(num_node_features=data.x.shape[1], 
                num_hidden=128,
                num_classes=(data.y.max()+1).item()
               ).to(device)
        
        elif model_type == 'appnp':
            model = APPNPNet(num_node_features=data.x.shape[1], 
                     num_hidden=64,
                     num_classes=(data.y.max()+1).item(),
                     K=10,
                     alpha=0.1).to(device)
        elif model_type == 'gsage':
            model = GraphSAGE(num_node_features=data.x.shape[1], 
                num_hidden=128,
                num_classes=(data.y.max()+1).item()
               ).to(device)
            
        try:
            model_path = f'./{model_type}_best_fold_{fold}.pt' 
            model.load_state_dict(torch.load(model_path))
            model.eval()

            with torch.no_grad():
                out = model(data.to(device))
                preds = out.argmax(dim=1)
                all_predictions.append(preds.cpu().numpy())
        except:
            continue

all_predictions = np.array(all_predictions)
majority_votes, _ = mode(all_predictions, axis=0)
majority_votes = torch.tensor(majority_votes.squeeze(), dtype=torch.long)
# correct = (majority_votes[data.val_mask] == data.y[data.val_mask]).sum()
# acc = int(correct) / int(data.val_mask.sum())
# print(f'Accuracy: {acc:.4f}')

In [19]:
# model.eval()
# pred = model(data).argmax(dim=1)
# correct = (pred[data.val_mask] == data.y[data.val_mask]).sum()
# acc = int(correct) / int(data.val_mask.sum())
# print(f'Accuracy: {acc:.4f}')

## Submitting the result

In [26]:
preds = majority_votes[idx_test]
np.savetxt('submission.txt', preds, fmt='%d')